#  Install TiRex package

In [1]:
!pip install 'tirex-ts[notebooks,gluonts,hfdataset,classification]' -q
!pip install aeon

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 25.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [5]:
!pip install 'git+https://github.com/NX-AI/tirex.git#egg=tirex-ts[classification]'


DEPRECATION: git+https://github.com/NX-AI/tirex.git#egg=tirex-ts[classification] contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617
  Cloning https://github.com/NX-AI/tirex.git to /tmp/pip-install-as73siom/tirex-ts_4b9c3d0476f84ea99655dd5541a9de72
  Running command git clone --filter=blob:none --quiet https://github.com/NX-AI/tirex.git /tmp/pip-install-as73siom/tirex-ts_4b9c3d0476f84ea99655dd5541a9de72
  Resolved https://github.com/NX-AI/tirex.git to commit c54719aab2807c6dbd6a367896ce34f5306795eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tirex-ts: filename=tirex_ts-1.4.0-py3-none-any.whl size=60527 sha256=a4b813087e6cb4791d3834114ba93ac7b8ce7883ee9aa78ef0d0aabd31749fae
  Stored in d

# Load Example Data

In [1]:
import torch
from aeon.datasets import load_italy_power_demand
from sklearn.preprocessing import LabelEncoder

# Load train data
train_X, train_y = load_italy_power_demand(split="train")

# Load test data
test_X, test_y = load_italy_power_demand(split="test")

# Encode string labels -> integers
label_encoder = LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.transform(test_y)

# Convert to torch tensors
train_X = torch.tensor(train_X, dtype=torch.float32)
test_X = torch.tensor(test_X, dtype=torch.float32)

train_y = torch.tensor(train_y, dtype=torch.long)
test_y = torch.tensor(test_y, dtype=torch.long)

print(train_X.shape, train_y.shape)
print(test_X.shape, test_y.shape)

torch.Size([67, 1, 24]) torch.Size([67])
torch.Size([1029, 1, 24]) torch.Size([1029])


# Initialize Embedding model and Classification Head

## 1. Random Forest Classifier

In [2]:
from tirex.models.classification import TirexRFClassifier

classifier = TirexRFClassifier(n_estimators=50, max_depth=10, random_state=42, batch_size=256)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.ckpt:   0%|          | 0.00/141M [00:00<?, ?B/s]

### Fit Model

In [3]:
fit_meta = classifier.fit((train_X, train_y))

### Prediction results

In [4]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))

Accuracy 0.9659863945578231
F1 Score 0.9659843385339878


## 2. Linear Regression Classifier

In [7]:
from tirex.models.classification import TirexLinearClassifier

classifier_2 = TirexLinearClassifier(data_augmentation=False, compile=False)

In [8]:
fit_meta_2 = classifier_2.fit((train_X, train_y))

Epoch 1, Train Loss: 0.747680, Val Loss: 0.678471
Epoch 6, Train Loss: 0.472694, Val Loss: 0.446795


In [9]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier_2.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))

Accuracy 0.8979591836734694
F1 Score 0.8978047578555883


## 3. Gradient Boosting Classifier

In [10]:
from tirex.models.classification import TirexGBMClassifier

classifier_3 = TirexGBMClassifier(data_augmentation=False, compile=False)

In [11]:
fit_meta_3 = classifier_3.fit((train_X, train_y))

[LightGBM] [Info] Number of positive: 26, number of negative: 27
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.010349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 118531
[LightGBM] [Info] Number of data points in the train set: 53, number of used features: 6144
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.490566 -> initscore=-0.037740
[LightGBM] [Info] Start training from score -0.037740
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [12]:
from sklearn.metrics import accuracy_score, f1_score

pred_y = classifier_3.predict(test_X)
pred_y = pred_y.cpu().numpy()

print("Accuracy " + str(accuracy_score(test_y, pred_y)))
print("F1 Score " + str(f1_score(test_y, pred_y, average="macro")))

Accuracy 0.956268221574344
F1 Score 0.9562667346678567


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
